### **Импорт библиотек**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost
!pip install tqdm
!pip install pymorphy2[fast]
!pip install fasttext

     |████████████████████████████████| 67.4 MB 26 kB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 16.1 MB/s 
     |████████████████████████████████| 371 kB 37.1 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=860122 sha256=8c8efb6cc90b92525dc74ac01a126cec6da5586a08ff181c8f516b490fcc8f9c
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG
     |████████████████████████████████| 68 kB 2.1 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3091910 sha256=63733d5d95752c6f61eb480d32d09cbb913df0f0e2754622657ff97afb075a51
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [3]:
import numpy as np
import pandas as pd
import re

from collections import Counter
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
import catboost as ctb
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import RidgeClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering
from sklearn import utils

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
from pymorphy2 import MorphAnalyzer
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

import gensim
import gensim.downloader as api
from gensim.test.utils import get_tmpfile
import fasttext
from gensim.models import FastText, KeyedVectors
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### **Считываем и смотрим на данные**

In [4]:
labled_train_data = pd.read_csv('/content/drive/MyDrive/labled_train_data.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
labled_train_comments = pd.read_csv('/content/drive/MyDrive/labled_train_comments.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
labled_train_speed = pd.read_csv('/content/drive/MyDrive/labled_train_tracks_speed.csv', comment='#', sep=',')
labled_train_tracks = pd.read_csv('/content/drive/MyDrive/labled_train_tracks.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)

unlabled_train_data = pd.read_csv('/content/drive/MyDrive/unlabled_train_data.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
unlabled_train_comments = pd.read_csv('/content/drive/MyDrive/unlabled_train_comments.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
unlabled_train_speed = pd.read_csv('/content/drive/MyDrive/unlabled_train_tracks_speed.csv', comment='#', sep=',')
unlabled_train_tracks = pd.read_csv('/content/drive/MyDrive/unlabled_train_tracks.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)

In [ ]:
labled_train_comments = pd.concat([labled_train_comments, unlabled_train_comments], ignore_index=True)

In [29]:
labled_train_data.head()

,order_id,driver_id,client_id,dttm,date,arrived_distance,arrived_duration,distance,duration,from_latitude,from_longitude,to_latitude,to_longitude,mark,client_rate_ride,client_rides_cnt,driver_rides_cnt,comment,is_aggressive
0,6a0f322ade1a05e5c4cec4344efbce8b,f7c2b293ef94420f5e51abae6889b83b,3156d05c6458a8228bed59f02075a61e,2021-01-22 21:53:00,2021-01-22,150.0,1.0,11.8,20.5,55.795900,37.560300,55.716502,37.524627,Kia K5,NaN,16.0,39.0,Больше нечего сказать,0
1,934ecbe5845426fd3f8ef7938cce2a11,01d029c42c99581080a60679fca06ff9,3156d05c6458a8228bed59f02075a61e,2021-01-24 14:09:00,2021-01-24,570.0,1.0,17.2,26.2,55.716502,37.524627,55.808253,37.638847,Volkswagen Polo,NaN,18.0,338.0,Да,0
2,5348cb339b63eaea3b2cb57a064ce550,3c88deb7df7a73a24ebc229db9783405,3156d05c6458a8228bed59f02075a61e,2021-01-26 21:02:00,2021-01-26,40.0,0.0,5.8,10.9,55.716637,37.524223,55.741958,37.568172,MercedesBenz EClass,NaN,17.0,162.0,Больше нечего сказать,0
3,309ef91c3b51e27d097642169576f67b,f35a8ff85f2095755f16bba91035fbdc,3156d05c6458a8228bed59f02075a61e,2021-01-27 17:24:00,2021-01-27,140.0,1.0,4.5,10.7,55.689076,37.491089,55.716502,37.524627,Kia Optima,NaN,18.0,136.0,Больше нечего сказать,0
4,3506e04e45d39c6e3033637389da1041,0a227ac8d702170c03acf36d55e60d0d,3156d05c6458a8228bed59f02075a61e,2021-01-29 15:31:00,2021-01-29,150.0,1.0,15.4,25.1,55.655489,37.616629,55.716502,37.524627,Kia Rio,5.0,20.0,194.0,Больше нечего сказать,0


### **Тест**

In [56]:
class Model_test:
  def __init__(self):
    self.model = None
    self.clustering_model = KMeans(n_clusters=3, random_state=42, n_init=50, max_iter=300)
    self.spectral_clusterer = SpectralClustering(assign_labels='discretize', n_clusters=3,
    random_state=42)
    self.cars_vectorizer = Doc2Vec(min_count=1, size=8, window=2, workers=-1, seed=42, negative=0)
    self.comm_model = LogisticRegression(random_state=42, max_iter=1000)
    self.cars_clusterer = KMeans(n_clusters=3, random_state=42, n_init=50, max_iter=300)
    self.text_vectorizer = Doc2Vec(min_count=1, size=30, window=5, workers=-1, seed=42, negative=3)
    
    self.standart_comments = ['Больше нечего сказать'.lower(), 'Да'.lower(), 'Ок'.lower()]
    self.aggressive_words = {'verb': set(), 'adj': set(), 'all_words': set(), 'noun': set()}
    self.morph_analyzer = MorphAnalyzer()

    self.stop_words = set(stopwords.words('russian')) # стоп-слова из nltk
    self.stop_words.add('')
    self.stop_words.add(' ')
    self.stop_words.add('\t')

  # возвращает нормальную форму слова(при normal_form=True, иначе просто слово) и его тэг(характеристики слова)
  def word_preprocess(self, word, word_normal_form=False):
      word = re.sub(r'[\d\W]', '', word).lower().strip() # убирает пробелы, цифры и знаки препинания
      word = word.replace('_', '')
      w = self.morph_analyzer.parse(word)[0]
      if word_normal_form:
        return w.normal_form, w.tag
      return word, w.tag

  def train_doc2vec_model(self, X, y, comm_dataset=None):
    print('training vectorizer model...')
    dataset = X.join(y).copy()
    dataset = dataset.fillna({'comment': self.standart_comments[0]})
    # dataset = dataset.loc[dataset['is_aggressive'] == 1]
    data = []
    tag_n = 0
    for row in dataset.itertuples(): # перебираем все строки в датасете
        comment = getattr(row, 'comment')
        comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        comment = [word for word in comment if word != '']
        if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          data.append(TaggedDocument(comment, [tag_n]))
          tag_n += 1
    if comm_dataset is not None:
      comm_dataset = comm_dataset.fillna({'comment': self.standart_comments[0]})
      for row in comm_dataset.itertuples():
        comment = getattr(row, 'comment')
        comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        comment = [word for word in comment if word != '']
        if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          data.append(TaggedDocument(comment, [tag_n]))
          tag_n += 1
    self.text_vectorizer.build_vocab(data)
    for epoch in range(30):
      self.text_vectorizer.train(utils.shuffle(data), total_examples=len(data), epochs=1)
      self.text_vectorizer.alpha -= 0.002
      self.text_vectorizer.min_alpha = self.text_vectorizer.alpha
    print('TEXT WECTORIZER TRAINED')
    
  # тренировка векторизатора машин
  def train_cars_vectorizer_and_clusterer(self, X, y):
      print('training vectorizer model...')
      dataset = X.join(y).copy()
      data = []
      tag_n = 0
      for row in dataset.itertuples(): # перебираем все строки в датасете
          if getattr(row, 'mark'):
            car = getattr(row, 'mark')
            car = [word.lower().strip() for word in car.split(" ")]
            car = [word for word in car if word != '']
            if len(car) > 0:
              data.append(TaggedDocument(car, [tag_n]))
              tag_n += 1
      self.cars_vectorizer.build_vocab(data)
      for epoch in range(20):
        self.cars_vectorizer.train(utils.shuffle(data), total_examples=len(data), epochs=1)
        self.cars_vectorizer.alpha -= 0.002
        self.cars_vectorizer.min_alpha = self.cars_vectorizer.alpha
      print('CARS WECTORIZER TRAINED')
      print('training cars clustering model...')
      vectors = []
      c = 0
      for row in dataset.itertuples(): # перебираем все строки в датасете
        if getattr(row, 'mark'):
          car = getattr(row, 'mark')
          car = [word.lower().strip() for word in car.split(" ")]
          car = [word for word in car if word != '']
          if len(car) > 0:
            vectors.append(self.cars_vectorizer.infer_vector(car))
      self.cars_clusterer.fit(vectors)
      print('CARS CLUSTERING COMPLETED')

  # генерация фич из кластеров
  def cluster_features(self, text):
    similarity = []
    clusters = self.clustering_model.cluster_centers_
    prep_text = [self.word_preprocess(word, word_normal_form=True)[0] for word in text.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
    prep_text = [word for word in text if word != '']
    if len(prep_text) == 0 or text.lower().strip() in self.standart_comments:
      for i in range(len(clusters)):
        similarity.append([0, len(clusters)])
      return similarity

    vector = self.text_vectorizer.infer_vector(prep_text)
    for i in range(len(clusters)):
      similarity.append([cosine(vector, clusters[i]), self.clustering_model.predict([vector])[0]])
    return similarity

  def train_comm_model(self, X, y):
    dataset = X.join(y).copy()
    dataset = dataset.fillna({'comment': self.standart_comments[0]})
    vectors = []
    c = 0
    train_y = []
    for row in dataset.itertuples(): # перебираем все строки в датасете
      comment = getattr(row, 'comment')
      comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
      comment = [word for word in comment if word != '']
      if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
        vectors.append(self.text_vectorizer.infer_vector(comment))
        train_y.append(getattr(row, 'is_aggressive'))
    self.clustering_model.fit(vectors)
    print('CLUSTERING COMPLETED')
    self.comm_model.fit(vectors, train_y)

  # средний рейтинг по комментариям для каждого водителя  (плохо работает, хз че с ними делать), не юзать пока
  def mean_comments_aggressive_rate(self, comm_dataset, X):
    mean_comments_aggressive_rate = []
    comm_dataset = comm_dataset.fillna({'comment': self.standart_comments[0]})
    for drv_id in X['driver_id']:
      driver_comments_rate = []
      driver_comments = comm_dataset.loc[comm_dataset['driver_id'] == drv_id]
      for row in driver_comments.itertuples():
        commentt = getattr(row, 'comment')
        commentt = [self.word_preprocess(word, word_normal_form=True)[0] for word in commentt.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        commentt = [word for word in commentt if word != '']
        if getattr(row, 'comment').lower().strip() not in self.standart_comments and len(commentt) > 0:
          driver_comments_rate.append(self.comm_model.predict_proba([self.text_vectorizer.infer_vector(commentt)])[:, 1][0])
        else:
          driver_comments_rate.append(0)
      if len(driver_comments_rate) == 0:
        driver_comments_rate.append(0)
      mean_comments_aggressive_rate.append(np.mean(driver_comments_rate))
    return mean_comments_aggressive_rate

  # заполнение словаря агрессивными словами
  def fill_agressive_vocab(self, X, y):
    dataset = X.join(y).copy()
    for row in dataset.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'is_aggressive') == 1 and getattr(row, 'comment') and getattr(row, 'comment').lower().strip() not in self.standart_comments:
        words = [self.word_preprocess(word, word_normal_form=True) for word in getattr(row, 'comment').split(' ')]
        for word in words:
          if word[0] not in self.stop_words:
            if 'VERB' in word[1]:
              self.aggressive_words['verb'].add(word[0])
            elif 'ADJF' in word[1] or 'ADJS' in word[1]:
              self.aggressive_words['adj'].add(word[0])
            elif 'NOUN' in word[1]:
              self.aggressive_words['noun'].add(word[0])

  # делаем NLP фичи на основе сгенерированного словаря
  def NLP_feature_extract(self, X, y=None):
    agg_verbs_rate = [] # глаголы
    agg_adjs_rate = [] # прилагательные
    agg_nouns_rate = [] # существительные
    for row in X.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'comment'):
        words = [self.word_preprocess(word, word_normal_form=True) for word in getattr(row, 'comment').split(' ')]
        words_verb = [word[0] for word in words if 'VERB' in word[1] and word[0] not in self.stop_words]
        words_adj = [word[0] for word in words if ('ADJF' in word[1] or 'ADJS' in word[1]) and word[0] not in self.stop_words]
        words_noun = [word[0] for word in words if 'NOUN' in word[1] and word[0] not in self.stop_words]

        if len(words_verb) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          agg_verbs_rate.append(len(set(words_verb) & self.aggressive_words['verb']) / len(words_verb))
        else:
          agg_verbs_rate.append(0)

        if len(words_adj) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          agg_adjs_rate.append(len(set(words_adj) & self.aggressive_words['adj']) / len(words_adj))
        else:
          agg_adjs_rate.append(0)

        if len(words_noun) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          agg_nouns_rate.append(len(set(words_noun) & self.aggressive_words['noun']) / len(words_noun))
        else:
          agg_nouns_rate.append(0)

    return agg_verbs_rate, agg_adjs_rate, agg_nouns_rate

  # отбор фичей
  def features(self, X, comm_dataset=None):
    data = X.copy()
    agg_verbs_rate, agg_adjs_rate, agg_nouns_rate = self.NLP_feature_extract(data)

    data['agg_verbs_rate'] = agg_verbs_rate
    data['agg_adjs_rate'] = agg_adjs_rate
    data['agg_nouns_rate'] = agg_nouns_rate

    feature_list = ['agg_verbs_rate', 'agg_adjs_rate', 'agg_nouns_rate']

    if comm_dataset is not None:
      data['mean_comments_aggressive_rate'] = self.mean_comments_aggressive_rate(comm_dataset, X)
      feature_list.append('mean_comments_aggressive_rate')
    
    probabilities = []
    for row in data.itertuples(): # перебираем все строки в датасете
       comment = getattr(row, 'comment')
       comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
       comment = [word for word in comment if word != '']
       if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
         probabilities.append(self.comm_model.predict_proba([self.text_vectorizer.infer_vector(comment)])[:, 1][0])
       else: 
         probabilities.append(0)

    data['agg_comm_probability'] = probabilities
    feature_list.append('agg_comm_probability')

    cars_cluster = []
    for row in data.itertuples(): # перебираем все строки в датасете
       car = getattr(row, 'mark')
       car = [word.lower().strip() for word in car.split(" ")]
       car = [word for word in car if word != '']
       cars_cluster.append(self.cars_clusterer.predict([self.cars_vectorizer.infer_vector(car)])[0])
    data['cars_cluster'] = cars_cluster
    feature_list.append('cars_cluster')

    # vectors = []
    # for row in data.itertuples(): # перебираем все строки в датасете
    #   comment = getattr(row, 'comment')
    #   comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
    #   comment = [word for word in comment if word != '']
    #   if len(comment) == 0:
    #     comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in self.standart_comments[0].split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
    #   vectors.append(self.text_vectorizer.infer_vector(comment))
    # data['spectral_clusters'] = self.spectral_clustering(vectors)
    # feature_list.append('spectral_clusters')

    # similarities = []
    # for row in data.itertuples(): # перебираем все строки в датасете
    #   comment = getattr(row, 'comment')
    #   similarities.append(self.cluster_features(comment))


    # for cluster in range(len(self.clustering_model.cluster_centers_)):
    #   data[f"euclidean_cluster_{cluster}"] = [similarities[i][cluster][0] for i in range(len(similarities))]
    #   feature_list.append(f"euclidean_cluster_{cluster}")
    # data['cluster'] = [similarities[i][0][1] for i in range(len(similarities))]
    # feature_list.append('cluster')

    data['is_comment'] = [1 if getattr(row, 'comment') and getattr(row, 'comment').lower().strip() not in self.standart_comments else 0 for row in data.itertuples()]
    feature_list.append('is_comment')

    # заполним NaN средними значениями
    for feature in feature_list:
      data = data.fillna({feature: data[feature].mean()})
    
    data = data.set_index('order_id')
    return data[feature_list]

  # кросс-валидация и предикт на тесте
  def train_eval(self, X, y, comm_dataset_labled=None, comm_dataset_unlabled=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    X_train = X_train.fillna({'comment': self.standart_comments[0]})
    X_test = X_test.fillna({'comment': self.standart_comments[0]})

    self.fill_agressive_vocab(X_train, y_train) # заполнение словаря агрессивных слов
    self.train_doc2vec_model(X_train, y_train, comm_dataset_labled) # тренировка doc2vec модели и кластеризации для комментариев о поездке
    self.train_cars_vectorizer_and_clusterer(X_train, y_train) # тренировка doc2vec модели и кластеризации для машин
    self.train_comm_model(X_train, y_train) # тренировка модели вероятностей агрессивности текстов

    X_train_features = self.features(X_train, comm_dataset_labled)

    self.model = LogisticRegression(random_state=42, max_iter=1000)

    cv_score = cross_val_score(self.model, X_train_features, y_train, cv=5, scoring='roc_auc')
    
    self.model.fit(X_train_features, y_train)

    print('Test Roc-Auc score:', roc_auc_score(y_test, self.model.predict_proba(self.features(X_test, comm_dataset_labled))[:, 1]))
    print('Train Roc-Auc score:', roc_auc_score(y_train, self.model.predict_proba(X_train_features)[:, 1]))
    print(f"CV_mean roc_auc: {np.mean(cv_score)}, CV_folds_score: {cv_score}")
    return X_train_features
  
  def predict(self, X, comm_dataset=None):
    res = self.model.predict_proba(self.features(X, comm_dataset))[:, 1]
    X = X.fillna({'comment': self.standart_comments[0]})
    datasss = pd.DataFrame()
    datasss['is_aggressive'] = res
    datasss.to_csv('resss.csv')
    print('prediction saved')

In [57]:

model_test = Model_test()
features = model_test.train_eval(X, y, comm_dataset_labled=None)

training vectorizer model...
TEXT WECTORIZER TRAINED
training vectorizer model...
CARS WECTORIZER TRAINED
training cars clustering model...
CARS CLUSTERING COMPLETED
CLUSTERING COMPLETED
Test Roc-Auc score: 0.6599572615941547
Train Roc-Auc score: 0.7110839103825374
CV_mean roc_auc: 0.7095085388788034, CV_folds_score: [0.70010653 0.71683099 0.70772684 0.68695851 0.73591982]


In [ ]:
model_test.predict(labled_test_data, comm_dataset=labled_train_comments)

prediction saved


In [58]:
features.head(100)

,agg_verbs_rate,agg_adjs_rate,agg_nouns_rate,agg_comm_probability,cars_cluster,is_comment
order_id,,,,,,
abc3159a76397972ed5bc025093ee6ee,0.0,0.0,0.000000,0.000000,0,0
8a79ddc85d97f0936e3de39622a8647e,0.0,0.0,0.000000,0.000000,2,0
1fb57420a9924e9bbae9768d113283c3,0.0,0.0,0.000000,0.000000,0,0
4d338112139fea37dcc4fa9c3d7ac14f,1.0,1.0,0.666667,0.162616,2,1
17968f160d85183f977a126297552deb,0.0,0.0,0.000000,0.000000,0,0
...,...,...,...,...,...,...
22e5f7df64bf76a0a3f521417871f169,0.0,0.0,0.500000,0.162957,1,1
8bb9be586bf1eeccffdfcdbe265fcbd3,0.0,0.0,0.000000,0.000000,1,0
830f2076f815ba5f9944c33880cfb4fa,0.0,0.0,0.000000,0.000000,0,0


### **Модель**

In [ ]:
labled_train_data = pd.read_csv('/content/drive/MyDrive/labled_train_data.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
labled_train_comments = pd.read_csv('/content/drive/MyDrive/labled_train_comments.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
labled_train_speed = pd.read_csv('/content/drive/MyDrive/labled_train_tracks_speed.csv', comment='#', sep=',')
labled_train_tracks = pd.read_csv('/content/drive/MyDrive/labled_train_tracks.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)

unlabled_train_data = pd.read_csv('/content/drive/MyDrive/unlabled_train_data.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
unlabled_train_comments = pd.read_csv('/content/drive/MyDrive/unlabled_train_comments.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
unlabled_train_speed = pd.read_csv('/content/drive/MyDrive/unlabled_train_tracks_speed.csv', comment='#', sep=',')
unlabled_train_tracks = pd.read_csv('/content/drive/MyDrive/unlabled_train_tracks.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)

In [5]:
X, y = labled_train_data[labled_train_data.columns[:-1]], labled_train_data['is_aggressive']

In [79]:
labled_test_data = pd.read_csv('/content/drive/MyDrive/labled_test_data.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)

In [76]:
class Model:
  def __init__(self):
    self.model = None
    self.cars_vectorizer = Doc2Vec(min_count=1, size=8, window=2, workers=-1, seed=42)
    self.comm_model = LogisticRegression(random_state=42, max_iter=1000)
    self.cars_clusterer = KMeans(n_clusters=4, random_state=42, n_init=50, max_iter=300)
    self.text_vectorizer = Doc2Vec(min_count=1, size=30, window=5, workers=-1, seed=42)
    
    self.standart_comments = ['Больше нечего сказать'.lower(), 'Да'.lower(), 'Ок'.lower()]
    self.aggressive_words = {'verb': set(), 'adj': set(), 'all_words': set(), 'noun': set()}
    self.morph_analyzer = MorphAnalyzer()

    self.stop_words = set(stopwords.words('russian')) # стоп-слова из nltk
    self.stop_words.add('')
    self.stop_words.add(' ')
    self.stop_words.add('\t')

  # возвращает нормальную форму слова(при normal_form=True, иначе просто слово) и его тэг(характеристики слова)
  def word_preprocess(self, word, word_normal_form=False):
      word = re.sub(r'[\d\W]', '', word).lower().strip() # убирает пробелы, цифры и знаки препинания
      word = word.replace('_', '')
      w = self.morph_analyzer.parse(word)[0]
      if word_normal_form:
        return w.normal_form, w.tag
      return word, w.tag

  def train_doc2vec_model(self, X, y, comm_dataset=None):
    print('training vectorizer model...')
    dataset = X.join(y).copy()
    dataset = dataset.fillna({'comment': self.standart_comments[0]})
    data = []
    tag_n = 0
    for row in dataset.itertuples(): # перебираем все строки в датасете
        comment = getattr(row, 'comment')
        comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        comment = [word for word in comment if word != '']
        if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          data.append(TaggedDocument(comment, [tag_n]))
          tag_n += 1
    if comm_dataset is not None:
      comm_dataset = comm_dataset.fillna({'comment': self.standart_comments[0]})
      for row in comm_dataset.itertuples():
        comment = getattr(row, 'comment')
        comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        comment = [word for word in comment if word != '']
        if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          data.append(TaggedDocument(comment, [tag_n]))
          tag_n += 1
    self.text_vectorizer.build_vocab(data)
    for epoch in range(30):
      self.text_vectorizer.train(utils.shuffle(data), total_examples=len(data), epochs=1)
      self.text_vectorizer.alpha -= 0.002
      self.text_vectorizer.min_alpha = self.text_vectorizer.alpha
    print('TEXT WECTORIZER TRAINED')
    
  # тренировка векторизатора машин
  def train_cars_vectorizer_and_clusterer(self, X, y):
      print('training vectorizer model...')
      dataset = X.join(y).copy()
      data = []
      tag_n = 0
      for row in dataset.itertuples(): # перебираем все строки в датасете
          if getattr(row, 'mark'):
            car = getattr(row, 'mark')
            car = [word.lower().strip() for word in car.split(" ")]
            car = [word for word in car if word != '']
            if len(car) > 0:
              data.append(TaggedDocument(car, [tag_n]))
              tag_n += 1
      self.cars_vectorizer.build_vocab(data)
      for epoch in range(20):
        self.cars_vectorizer.train(utils.shuffle(data), total_examples=len(data), epochs=1)
        self.cars_vectorizer.alpha -= 0.002
        self.cars_vectorizer.min_alpha = self.cars_vectorizer.alpha
      print('CARS WECTORIZER TRAINED')
      print('training cars clustering model...')
      vectors = []
      c = 0
      for row in dataset.itertuples(): # перебираем все строки в датасете
        if getattr(row, 'mark'):
          car = getattr(row, 'mark')
          car = [word.lower().strip() for word in car.split(" ")]
          car = [word for word in car if word != '']
          if len(car) > 0:
            vectors.append(self.cars_vectorizer.infer_vector(car))
      self.cars_clusterer.fit(vectors)
      print('CARS CLUSTERING COMPLETED')

  def train_comm_model(self, X, y):
    dataset = X.join(y).copy()
    dataset = dataset.fillna({'comment': self.standart_comments[0]})
    vectors = []
    c = 0
    train_y = []
    for row in dataset.itertuples(): # перебираем все строки в датасете
      comment = getattr(row, 'comment')
      comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
      comment = [word for word in comment if word != '']
      if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
        vectors.append(self.text_vectorizer.infer_vector(comment))
        train_y.append(getattr(row, 'is_aggressive'))
    self.comm_model.fit(vectors, train_y)
    print('COMM MODEL TRAINED')

  # средний рейтинг по комментариям для каждого водителя  (плохо работает, хз че с ними делать), не юзать пока
  def mean_comments_aggressive_rate(self, comm_dataset, X):
    mean_comments_aggressive_rate = []
    comm_dataset = comm_dataset.fillna({'comment': self.standart_comments[0]})
    for drv_id in X['driver_id']:
      driver_comments_rate = []
      driver_comments = comm_dataset.loc[comm_dataset['driver_id'] == drv_id]
      for row in driver_comments.itertuples():
        commentt = getattr(row, 'comment')
        commentt = [self.word_preprocess(word, word_normal_form=True)[0] for word in commentt.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        commentt = [word for word in commentt if word != '']
        if getattr(row, 'comment').lower().strip() not in self.standart_comments and len(commentt) > 0:
          driver_comments_rate.append(self.comm_model.predict_proba([self.text_vectorizer.infer_vector(commentt)])[:, 1][0])
        else:
          driver_comments_rate.append(0)
      if len(driver_comments_rate) == 0:
        driver_comments_rate.append(0)
      mean_comments_aggressive_rate.append(np.mean(driver_comments_rate))
    return mean_comments_aggressive_rate

  # заполнение словаря агрессивными словами
  def fill_agressive_vocab(self, X, y):
    dataset = X.join(y).copy()
    for row in dataset.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'is_aggressive') == 1 and getattr(row, 'comment') and getattr(row, 'comment').lower().strip() not in self.standart_comments:
        words = [self.word_preprocess(word, word_normal_form=True) for word in getattr(row, 'comment').split(' ')]
        for word in words:
          if word[0] not in self.stop_words:
            if 'VERB' in word[1]:
              self.aggressive_words['verb'].add(word[0])
            elif 'ADJF' in word[1] or 'ADJS' in word[1]:
              self.aggressive_words['adj'].add(word[0])
            elif 'NOUN' in word[1]:
              self.aggressive_words['noun'].add(word[0])

  # делаем NLP фичи на основе сгенерированного словаря
  def NLP_feature_extract(self, X, y=None):
    agg_verbs_rate = [] # глаголы
    agg_adjs_rate = [] # прилагательные
    agg_nouns_rate = [] # существительные
    for row in X.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'comment'):
        words = [self.word_preprocess(word, word_normal_form=True) for word in getattr(row, 'comment').split(' ')]
        words_verb = [word[0] for word in words if 'VERB' in word[1] and word[0] not in self.stop_words]
        words_adj = [word[0] for word in words if ('ADJF' in word[1] or 'ADJS' in word[1]) and word[0] not in self.stop_words]
        words_noun = [word[0] for word in words if 'NOUN' in word[1] and word[0] not in self.stop_words]

        if len(words_verb) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          agg_verbs_rate.append(len(set(words_verb) & self.aggressive_words['verb']) / len(words_verb))
        else:
          agg_verbs_rate.append(0)

        if len(words_adj) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          agg_adjs_rate.append(len(set(words_adj) & self.aggressive_words['adj']) / len(words_adj))
        else:
          agg_adjs_rate.append(0)

        if len(words_noun) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
          agg_nouns_rate.append(len(set(words_noun) & self.aggressive_words['noun']) / len(words_noun))
        else:
          agg_nouns_rate.append(0)

    return agg_verbs_rate, agg_adjs_rate, agg_nouns_rate

  # отбор фичей
  def features(self, X, comm_dataset=None):
    data = X.copy()
    agg_verbs_rate, agg_adjs_rate, agg_nouns_rate = self.NLP_feature_extract(data)

    data['agg_verbs_rate'] = agg_verbs_rate
    data['agg_adjs_rate'] = agg_adjs_rate
    data['agg_nouns_rate'] = agg_nouns_rate

    feature_list = ['agg_verbs_rate', 'agg_adjs_rate', 'agg_nouns_rate']

    if comm_dataset is not None:
      data['mean_comments_aggressive_rate'] = self.mean_comments_aggressive_rate(comm_dataset, X)
      feature_list.append('mean_comments_aggressive_rate')
    
    probabilities = []
    for row in data.itertuples(): # перебираем все строки в датасете
       comment = getattr(row, 'comment')
       comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
       comment = [word for word in comment if word != '']
       if len(comment) > 0 and getattr(row, 'comment').lower().strip() not in self.standart_comments:
         probabilities.append(self.comm_model.predict_proba([self.text_vectorizer.infer_vector(comment)])[:, 1][0])
       else: 
         probabilities.append(0)

    data['agg_comm_probability'] = probabilities
    feature_list.append('agg_comm_probability')

    cars_cluster = []
    for row in data.itertuples(): # перебираем все строки в датасете
       car = getattr(row, 'mark')
       car = [word.lower().strip() for word in car.split(" ")]
       car = [word for word in car if word != '']
       cars_cluster.append(self.cars_clusterer.predict([self.cars_vectorizer.infer_vector(car)])[0])
    data['cars_cluster'] = cars_cluster
    feature_list.append('cars_cluster')

    data['is_comment'] = [1 if getattr(row, 'comment') and getattr(row, 'comment').lower().strip() not in self.standart_comments else 0 for row in data.itertuples()]
    feature_list.append('is_comment')

    # заполним NaN средними значениями
    for feature in feature_list:
      data = data.fillna({feature: data[feature].mean()})
    
    data = data.set_index('order_id')
    return data[feature_list]

  # кросс-валидация и предикт на тесте
  def train_eval(self, X, y, comm_dataset_labled=None, comm_dataset_unlabled=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    X_train = X_train.fillna({'comment': self.standart_comments[0]})
    X_test = X_test.fillna({'comment': self.standart_comments[0]})

    self.fill_agressive_vocab(X_train, y_train) # заполнение словаря агрессивных слов
    self.train_doc2vec_model(X_train, y_train, comm_dataset_labled) # тренировка doc2vec модели и кластеризации для комментариев о поездке
    self.train_cars_vectorizer_and_clusterer(X_train, y_train) # тренировка doc2vec модели и кластеризации для машин
    self.train_comm_model(X_train, y_train) # тренировка модели вероятностей агрессивности текстов

    X_train_features = self.features(X_train, comm_dataset_labled)

    self.model = LogisticRegression(random_state=42, max_iter=1000)

    cv_score = cross_val_score(self.model, X_train_features, y_train, cv=5, scoring='roc_auc')
    
    self.model.fit(X_train_features, y_train)

    print('Test Roc-Auc score:', roc_auc_score(y_test, self.model.predict_proba(self.features(X_test, comm_dataset_labled))[:, 1]))
    print('Train Roc-Auc score:', roc_auc_score(y_train, self.model.predict_proba(X_train_features)[:, 1]))
    print(f"CV_mean roc_auc: {np.mean(cv_score)}, CV_folds_score: {cv_score}")
    return X_train_features
  
  def predict(self, X, comm_dataset=None):
    res = self.model.predict_proba(self.features(X, comm_dataset))[:, 1]
    X = X.fillna({'comment': self.standart_comments[0]})
    datasss = pd.DataFrame()
    datasss['is_aggressive'] = res
    datasss.to_csv('resss.csv')
    print('prediction saved')

In [77]:
model = Model()
features = model.train_eval(X, y, comm_dataset_labled=None)

training vectorizer model...
TEXT WECTORIZER TRAINED
training vectorizer model...
CARS WECTORIZER TRAINED
training cars clustering model...
CARS CLUSTERING COMPLETED
COMM MODEL TRAINED
Test Roc-Auc score: 0.6556009300689736
Train Roc-Auc score: 0.7032904393750704
CV_mean roc_auc: 0.7018549220542715, CV_folds_score: [0.66870914 0.73824342 0.66279588 0.72440212 0.71512406]


In [82]:
features.head(40)

,agg_verbs_rate,agg_adjs_rate,agg_nouns_rate,agg_comm_probability,cars_cluster,is_comment
order_id,,,,,,
4a8adfc877d14e2d49d465c1bde8c839,0.0,0.0,0.000000,0.000000,0,0
b3457ff4fd872aa4afce47c482699d78,0.0,0.0,0.000000,0.000000,1,0
087ee63278c5468057f0da6f231527a7,0.0,0.0,0.000000,0.000000,1,0
1110046556247e4f5e2ccb255f6fe294,0.0,0.0,0.000000,0.000000,0,0
34b5c15740d45d8c3f4244ca7302fb33,0.0,0.0,0.000000,0.000000,1,0
bf1923849c85cc892b2fc9abd91342e5,0.0,0.0,0.000000,0.000000,2,0
576c627ebcc4b09fcc34f4c4bf40dcd5,0.0,0.0,0.000000,0.000000,2,0
e75c9a3569dcbacfb5c112bc6d5be2b9,0.0,0.0,0.000000,0.154038,3,1
2b3ebd4d7a1b7bf92a3316216e2172a9,0.0,0.0,0.000000,0.000000,2,0


### **Отсчет по фичам**

Скор без NLP фич - 0.5644
Скор с NLP фичами:

  - эмпирически: 0.6965 (если добавить все слова, а не по типу слов - результат хуже на 0.005) CV_mean: 0.7213110197676796, CV_std: 0.036754591689793115 - глаг+прил+сущ (но словарь сформирован для данных, на которых и идет предсказание, поэтому мб и такой скор)

  - кластеризация и семантическая модель (doc2vec): 0.640-651 (фичи тип кластера, евклидово расстояние до каждого кластера) CV_mean roc_auc: 0.6625926120596789, CV_folds_score: [0.63023583 0.71938044 0.62881106 0.68139226 0.65314347] косинусное расстояние хоть обычно и юзают для текстов, но тут ухудшило результат(странно)
  - эмпирические + кластеризация 0.668 CV_mean roc_auc: 0.6819522032387957, CV_folds_score: [0.62861001 0.74803329 0.66013863 0.70424985 0.66872923]

  